In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f

### One-way anova

In [45]:
def create_multiindex_df(*matrices):
    # Check if there are any matrices provided
    if not matrices:
        return pd.DataFrame()

    # Determine the total number of columns (assuming all matrices have the same number of columns)
    total_columns = len(matrices[0][0])

    # Creating single-level column names
    column_names = [f'Factor B Level {i+1}' for i in range(total_columns)]

    # Initialize an empty list to store the combined data and row indices
    combined_data = []
    row_indices = []

    # Current row number starts at 1
    current_row_num = 1

    for level, matrix in enumerate(matrices, start=1):
        level_name = f'Level {level}'
        for row in matrix:
            combined_data.append(row)
            row_indices.append((level_name, current_row_num))
            current_row_num += 1

    # Creating row multi-index
    row_index = pd.MultiIndex.from_tuples(row_indices, names=['Factor A', None])

    # Create DataFrame
    df = pd.DataFrame(combined_data, index=row_index, columns=column_names)

    display(df)
    
    return df

# Create and display the DataFrame
df = create_multiindex_df(factor_a_level_1, factor_a_level_2, factor_a_level_3)

Factor B Level 1  Factor B Level 2  Factor B Level 3  \
Factor A                                                           
Level 1  1                 1                 2                 3   
         2                 4                 5                 6   
Level 2  3                 7                 8                 9   
         4                10                11                12   
         5                13                14                15   
Level 3  6                 7                 8                 9   
         7                10                11                12   
         8                13                14                15   

            Factor B Level 4  
Factor A                      
Level 1  1                 1  
         2                 1  
Level 2  3                 1  
         4                 1  
         5                 1  
Level 3  6                 1  
         7                 1  
         8                 1

In [5]:
data = {
    'Factor A': ['Subjects'] * 7 + ['Subjects'] * 7 + ['Subjects'] * 7,
    'Factor B level 1': [9, 8, 7, 8, 8, 9, 8],
    'Factor B level 2': [7, 6, 6, 7, 8, 7, 6],
    'Factor B level 3': [4, 3, 2, 3, 4, 3, 2]
}

In [6]:
def perform_one_way_anova(data, id_var, value_vars):
    # Transforming data
    df = pd.DataFrame(data)
    df_melt = pd.melt(df, id_vars=[id_var], value_vars=value_vars, var_name='Factor', value_name='Score')

    # Constructing the formula for the ANOVA model
    formula = 'Score ~ C(Factor)'

    # One-way ANOVA
    model = ols(formula, data=df_melt).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)

    # Calculating MS
    anova_table['MS'] = anova_table['sum_sq'] / anova_table['df']

    # F threshold
    alpha = 0.05
    anova_table['F threshold'] = anova_table.apply(
        lambda row: f.ppf(1 - alpha, row['df'], anova_table.loc['Residual', 'df'])
        if 'C(Factor)' in row.name else '-', axis=1)

    # Renaming columns for display
    anova_table = anova_table.rename(columns={'sum_sq': 'SS', 'F': 'F experiment'})
    anova_table = anova_table[['SS', 'df', 'MS', 'F threshold', 'F experiment']]
    anova_table = anova_table.fillna('-')

    # Calculating and adding Total SS and df
    total_ss = anova_table['SS'].sum()
    total_df = df_melt.shape[0] - 1
    anova_table.loc['Total'] = [total_ss, total_df, '-', '-', '-']

    return anova_table

In [7]:
anova_table = perform_one_way_anova(data, 'Factor A', ['Factor B level 1', 'Factor B level 2', 'Factor B level 3'])
display(anova_table)

ValueError: All arrays must be of the same length

In [5]:
def perform_one_way_anova_from_scratch(data, id_var, value_vars):
    # Transforming data
    df = pd.DataFrame(data)
    df_melt = pd.melt(df, id_vars=[id_var], value_vars=value_vars, var_name='Factor', value_name='Score')

    # Overall mean
    grand_mean = df_melt['Score'].mean()

    # Sums of squares
    ss_total = sum((df_melt['Score'] - grand_mean)**2)
    ss_factor = sum(df_melt.groupby('Factor')['Score'].apply(lambda x: len(x) * (x.mean() - grand_mean)**2))
    ss_error = ss_total - ss_factor

    # Degrees of freedom
    df_factor = len(df_melt['Factor'].unique()) - 1
    df_error = len(df_melt) - len(df_melt['Factor'].unique())
    df_total = len(df_melt) - 1

    # Mean squares
    ms_factor = ss_factor / df_factor
    ms_error = ss_error / df_error

    # F-value
    f_factor = ms_factor / ms_error

    # F critical value for alpha = 0.05
    alpha = 0.05
    f_critical_factor = f.ppf(1 - alpha, df_factor, df_error)

    # Assembling the ANOVA table
    anova_table = pd.DataFrame({
        'SS': [ss_factor, ss_error, ss_total],
        'df': [df_factor, df_error, df_total],
        'MS': [ms_factor, ms_error, "-"],
        'F critical': [f_critical_factor, "-", "-"],
        'F': [f_factor, "-", "-"]        
    }, index=['Factor', 'Error', 'Total'])

    return anova_table.fillna('-')

# Example usage
data = {
    'Factor A': ['Subjects'] * 7,
    'Factor B level 1': [9, 8, 7, 8, 8, 9, 8],
    'Factor B level 2': [7, 6, 6, 7, 8, 7, 6],
    'Factor B level 3': [4, 3, 2, 3, 4, 3, 2]
}

In [6]:
anova_table = perform_one_way_anova_from_scratch(data, 'Factor A', ['Factor B level 1', 'Factor B level 2', 'Factor B level 3'])
display(anova_table)

,SS,df,MS,F critical,F
Factor,98.666667,2,49.333333,3.554557,86.333333
Error,10.285714,18,0.571429,-,-
Total,108.952381,20,-,-,-


In [7]:
data = {
    'School': ['High school students'] * 6 + ['College students'] * 6,
    'Week1': [3, 4, 5, 3, 4, 3, 1, 2, 1, 1, 1, 2],
    'Week2': [5, 4, 3, 5, 5, 5, 5, 4, 3, 5, 5, 4],
    'Week3': [7, 8, 7, 8, 7, 7, 9, 8, 9, 8, 7, 9]
}

##### Two-way anova for independent groups

In [8]:
def perform_two_way_anova(data, id_var, value_vars):
    # Transforming data
    df = pd.DataFrame(data)
    df_melt = pd.melt(df, id_vars=[id_var], value_vars=value_vars, var_name='Factor', value_name='Score')

    # Constructing the formula for the ANOVA model
    formula = 'Score ~ C({}) + C(Factor) + C({}):C(Factor)'.format(id_var, id_var)

    # Two-way ANOVA
    model = ols(formula, data=df_melt).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)

    # Calculating MS
    anova_table['MS'] = anova_table['sum_sq'] / anova_table['df']

    # F threshold
    alpha = 0.05
    anova_table['F critical'] = anova_table.apply(
        lambda row: f.ppf(1 - alpha, row['df'], anova_table.loc['Residual', 'df'])
        if 'C({})'.format(id_var) in row.name or 'C(Factor)' in row.name else '-', axis=1)

    # Renaming columns for display
    anova_table = anova_table.rename(columns={'sum_sq': 'SS'})
    anova_table = anova_table[['SS', 'df', 'MS', 'F critical', 'F']]
    anova_table = anova_table.fillna('-')

    # Calculating and adding Total SS and df
    total_ss = anova_table['SS'].sum()
    total_df = df_melt.shape[0] - 1
    anova_table.loc['Total'] = [total_ss, total_df, '-', '-', '-']

    return anova_table

In [9]:
anova_table = perform_two_way_anova(data, 'School', ['Week1', 'Week2', 'Week3'])
display(anova_table)

,SS,df,MS,F critical,F
C(School),2.250000,1.0,2.25,4.170877,4.175258
C(Factor),175.166667,2.0,87.583333,3.31583,162.525773
C(School):C(Factor),17.166667,2.0,8.583333,3.31583,15.927835
Residual,16.166667,30.0,0.538889,-,-
Total,210.750000,35.0,-,-,-


In [10]:
def two_way_anova_from_scratch(data, id_var, value_vars):
    # Transforming data
    df = pd.DataFrame(data)
    df_melt = pd.melt(df, id_vars=[id_var], value_vars=value_vars, var_name='Factor', value_name='Score')

    # Overall mean
    grand_mean = df_melt['Score'].mean()

    # Sums of squares
    ss_total = sum((df_melt['Score'] - grand_mean)**2)

    # Sum of squares for main effects
    ss_id_var = sum(df_melt.groupby(id_var)['Score'].apply(lambda x: len(x) * (x.mean() - grand_mean)**2))
    ss_factor = sum(df_melt.groupby('Factor')['Score'].apply(lambda x: len(x) * (x.mean() - grand_mean)**2))

    # Sum of squares for interaction
    ss_interaction = 0
    for id_value in df_melt[id_var].unique():
        for factor in df_melt['Factor'].unique():
            group_mean = df_melt[(df_melt[id_var] == id_value) & (df_melt['Factor'] == factor)]['Score'].mean()
            expected_mean = df_melt[df_melt[id_var] == id_value]['Score'].mean() + df_melt[df_melt['Factor'] == factor]['Score'].mean() - grand_mean
            ss_interaction += len(df_melt[(df_melt[id_var] == id_value) & (df_melt['Factor'] == factor)]) * (group_mean - expected_mean)**2

    # Sum of squares for error
    ss_error = ss_total - ss_id_var - ss_factor - ss_interaction

    # Degrees of freedom
    df_id_var = len(df[id_var].unique()) - 1
    df_factor = len(df_melt['Factor'].unique()) - 1
    df_interaction = df_id_var * df_factor
    df_error = len(df_melt) - (len(df[id_var].unique()) * len(df_melt['Factor'].unique()))
    df_total = len(df_melt) - 1

    # Mean squares
    ms_id_var = ss_id_var / df_id_var
    ms_factor = ss_factor / df_factor
    ms_interaction = ss_interaction / df_interaction
    ms_error = ss_error / df_error

    # F-values
    f_id_var = ms_id_var / ms_error
    f_factor = ms_factor / ms_error
    f_interaction = ms_interaction / ms_error

    # F critical value for alpha = 0.05
    alpha = 0.05
    f_critical_id_var = f.ppf(1 - alpha, df_id_var, df_error)
    f_critical_factor = f.ppf(1 - alpha, df_factor, df_error)
    f_critical_interaction = f.ppf(1 - alpha, df_interaction, df_error)

    # Assembling the ANOVA table
    anova_table = pd.DataFrame({
        'SS': [ss_id_var, ss_factor, ss_interaction, ss_error, ss_total],
        'df': [df_id_var, df_factor, df_interaction, df_error, df_total],
        'MS': [ms_id_var, ms_factor, ms_interaction, ms_error, "-"],
        'F critical': [f_critical_id_var, f_critical_factor, f_critical_interaction, "-", "-"],
        'F': [f_id_var, f_factor, f_interaction, "-", "-"]        
    }, index=[id_var, 'Factor', '{}:Factor'.format(id_var), 'Error', 'Total'])

    return anova_table.fillna('-')

In [11]:
anova_table = two_way_anova_from_scratch(data, 'School', ['Week1', 'Week2', 'Week3'])
display(anova_table)

,SS,df,MS,F critical,F
School,2.250000,1,2.25,4.170877,4.175258
Factor,175.166667,2,87.583333,3.31583,162.525773
School:Factor,17.166667,2,8.583333,3.31583,15.927835
Error,16.166667,30,0.538889,-,-
Total,210.750000,35,-,-,-


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import f

def mixed_design_anova(data, between_var, within_var):
    # Transforming data
    df = pd.DataFrame(data)
    df_melt = pd.melt(df, id_vars=[between_var], value_vars=within_var, var_name='Time', value_name='Score')

    # Overall mean
    grand_mean = df_melt['Score'].mean()

    # Sums of squares
    ss_between = sum(df_melt.groupby(between_var)['Score'].apply(lambda x: len(x) * (x.mean() - grand_mean)**2))
    ss_within = sum((df_melt['Score'] - df_melt.groupby([between_var, 'Time'])['Score'].transform('mean'))**2)
    ss_interaction = sum(df_melt.groupby([between_var, 'Time'])['Score'].apply(lambda x: len(x) * (x.mean() - df_melt[df_melt['Time'] == x.name[1]]['Score'].mean() - df_melt[df_melt[between_var] == x.name[0]]['Score'].mean() + grand_mean)**2))
    ss_total = sum((df_melt['Score'] - grand_mean)**2)
    ss_error = ss_total - ss_between - ss_within - ss_interaction

    # Degrees of freedom
    n_groups = df[between_var].nunique()
    n_measurements = len(within_var)
    n_total = len(df_melt)

    df_between = n_groups - 1
    df_within = n_total - n_groups
    df_interaction = df_between * (n_measurements - 1)
    df_error = n_total - n_groups * n_measurements
    df_total = n_total - 1

    # Mean squares
    ms_between = ss_between / df_between
    ms_within = ss_within / df_within
    ms_interaction = ss_interaction / df_interaction
    ms_error = ss_error / df_error

    # F-values
    f_between = ms_between / ms_within
    f_interaction = ms_interaction / ms_error

    # F critical value for alpha = 0.05
    alpha = 0.05
    f_critical_between = f.ppf(1 - alpha, df_between, df_within)
    f_critical_interaction = f.ppf(1 - alpha, df_interaction, df_error)

    # Assembling the ANOVA table
    anova_table = pd.DataFrame({
        'SS': [ss_between, ss_within, ss_interaction, ss_error, ss_total],
        'df': [df_between, df_within, df_interaction, df_error, df_total],
        'MS': [ms_between, ms_within, ms_interaction, ms_error, "-"],
        'F': [f_between, "-", f_interaction, "-", "-"],
        'F critical': [f_critical_between, "-", f_critical_interaction, "-", "-"]
    }, index=[f'{between_var} (A)', 'Error (S/A)', 'Interaction (AxB)', 'Error (BxS/A)', 'Total'])

    return anova_table.fillna('-')

data = {
    'School': ['High school students'] * 6 + ['College students'] * 6,
    'Week1': [3, 4, 5, 3, 4, 3, 1, 2, 1, 1, 1, 2],
    'Week2': [5, 4, 3, 5, 5, 5, 5, 4, 3, 5, 5, 4],
    'Week3': [7, 8, 7, 8, 7, 7, 9, 8, 9, 8, 7, 9]
}

anova_table = mixed_design_anova(data, 'School', ['Week1', 'Week2', 'Week3'])
print(anova_table)

                           SS  df        MS         F F critical
School (A)           2.250000   1      2.25  4.731959   4.130018
Error (S/A)         16.166667  34   0.47549         -          -
Interaction (AxB)   17.166667   2  8.583333  1.470029    3.31583
Error (BxS/A)      175.166667  30  5.838889         -          -
Total              210.750000  35         -         -          -
